In [1]:
import os
import matplotlib.pyplot as plt
from OA_utils.OAPreprocessingScripts import *
from OA_utils.OpenSimScripts import *
from OA_utils.data_utils import interp_segments, ad2float
import pickle
from collections import Counter

In [2]:
root_dir = '/Users/briankeller/Desktop/GRFMuscleModel/Old_Young_Walking_Data/'
subjects = [f'Y{i}' for i in range(1, 23)]
bad_subjects = ['Y3']
for bs in bad_subjects:
    subjects.remove(bs)
speeds = ['80', '100', '120']
n_trials = 5
subject_trials = {}
trial_names = []
for subj in subjects:
    subj_dir = os.path.join(root_dir, subj, 'Walking/Files_W_HJCs/')
    output_dir = os.path.join(root_dir, 'transformed/')
    # Initialize nested dict
    subject_trials[subj] = {
        'static': {
            'input': os.path.join(subj_dir, f'{subj}_walking_static1.trc'),
            'output': os.path.join(output_dir, f'{subj}_walk_static1_transformed.trc')
        },
        'tracking': [],
        'forces': [],
        
    }
    # Populate tracking and force trial lists
    for spd in speeds:
        for i in range(1, n_trials + 1):
            trial_name = f'{subj}_{spd}_{i}'
            trial_names.append(str(trial_name))
            tracking_in = os.path.join(subj_dir, f'{trial_name}.trc')
            tracking_out = os.path.join(output_dir, f'{trial_name}_transformed.trc')
            force_in = os.path.join(subj_dir, f'{trial_name}.forces')
            force_out = os.path.join(output_dir, f'{trial_name}_transformed.mot')

            subject_trials[subj]['tracking'].append({'input': tracking_in, 'output': tracking_out})
            subject_trials[subj]['forces'].append({'input': force_in, 'output': force_out})

all_segs = {}
for subj, data in subject_trials.items():
    process_hjc_trc(input_path=data['static']['input'], 
                    output_path=data['static']['output'], 
                    markers_to_drop=[])
    for trc, forces in zip(data['tracking'], data['forces']):
        trial_segs = preprocess_trc_grf(trc_ip=trc['input'], 
                           trc_op=trc['output'],
                           markers_to_drop=[], 
                           grf_ip=forces['input'], 
                           grf_op=forces['output'],
                           grf_pickle_path='')
        for trial_name, seg_dict in trial_segs.items():
            subj_name = trial_name.split('_')[0]  
            if subj_name not in all_segs:
                all_segs[subj_name] = {}
            all_segs[subj_name][trial_name] = seg_dict
print(all_segs)

{'Y1': {'Y1_80_1': {'left': [(1.4935, 2.3385)], 'right': [(0.859, 1.675), (2.168, 3.064)]}, 'Y1_80_2': {'left': [(1.7495, 2.676)], 'right': [(1.0535, 1.939), (2.4895, 3.4355)]}, 'Y1_80_3': {'left': [(1.8275, 2.7055)], 'right': [(1.1395, 2.016), (2.5135, 3.3855)]}, 'Y1_80_4': {'left': [(1.85, 2.719)], 'right': [(1.1705, 2.0375), (2.518, 3.3595)]}, 'Y1_80_5': {'left': [(1.658, 2.5195)], 'right': [(1.01, 1.84), (2.3465, 3.1985)]}, 'Y1_100_1': {'left': [(1.467, 2.1545)], 'right': [(0.912, 1.6065), (2.0385, 2.647)]}, 'Y1_100_2': {'left': [(1.396, 2.075)], 'right': [(0.9215, 1.5135), (1.9435, 2.625)]}, 'Y1_100_3': {'left': [(1.4125, 2.1075)], 'right': [(0.8525, 1.5535), (1.9755, 2.6685)]}, 'Y1_100_4': {'left': [(1.496, 2.106)], 'right': [(0.9505, 1.623), (2.021, 2.5625)]}, 'Y1_100_5': {'left': [(1.4145, 2.104)], 'right': [(0.8645, 1.551), (1.967, 2.645)]}, 'Y1_120_1': {'left': [(0.8535, 1.4955)], 'right': [(1.385, 2.0235)]}, 'Y1_120_2': {'left': [(1.2575, 1.899)], 'right': [(0.732, 1.3695)]}

In [4]:
subj_masses = [63, 85, 54, 60, 54, 83, 53, 62, 72, 65, 84, 83, 64, 59, 55, 73, 90, 75, 65, 68]
root_dir = '/Users/briankeller/Desktop/GRFMuscleModel/Old_Young_Walking_Data/'
for subj, data in subject_trials.items():
    scale_generic(root_dir= root_dir, mass=subj_masses[i], static_pose_filename=data['static']['output'])

[info] Processing subject Y1_scaled...
[info] Step 1: Loading generic model
[info] Updating Model file from 40000 to latest format...
[info] Loaded model RajagopalModifiedGeneric from file /Users/briankeller/Desktop/GRFMuscleModel/Old_Young_Walking_Data/Models/RajagopalModified_generic.osim
[warning] Couldn't find file 'r_pelvis.vtp'.
[warning] Couldn't find file 'l_pelvis.vtp'.
[warning] Couldn't find file 'sacrum.vtp'.
[warning] Couldn't find file 'r_femur.vtp'.
[warning] Couldn't find file 'r_tibia.vtp'.
[warning] Couldn't find file 'r_fibula.vtp'.
[warning] Couldn't find file 'r_patella.vtp'.
[warning] Couldn't find file 'r_talus.vtp'.
[warning] Couldn't find file 'r_foot.vtp'.
[warning] Couldn't find file 'r_bofoot.vtp'.
[warning] Couldn't find file 'l_femur.vtp'.
[warning] Couldn't find file 'l_tibia.vtp'.
[warning] Couldn't find file 'l_fibula.vtp'.
[warning] Couldn't find file 'l_patella.vtp'.
[warning] Couldn't find file 'l_talus.vtp'.
[warning] Couldn't find file 'l_foot.vtp'

In [7]:
parsed = parse_combined_scaling_output('/Users/briankeller/Desktop/GRFMuscleModel/Old_Young_Walking_Data/Y_scaling_log.txt')
for subject, info in parsed.items():
    rms = info['marker_error_rms']
    max = info['marker_error_max']
    marker = info['marker_error_max_marker']
    print(f'{subject} had marker error rms {rms} and max marker error {max} at the {marker} marker')

Read scaling log
Y1 had marker error rms 0.0209464 and max marker error 0.0353988 at the R.ASIS marker
Y2 had marker error rms 0.017833 and max marker error 0.030939 at the L.ASIS marker
Y4 had marker error rms 0.0158646 and max marker error 0.024268 at the R.Knee marker
Y5 had marker error rms 0.0191026 and max marker error 0.0352803 at the R.Knee marker
Y6 had marker error rms 0.0131734 and max marker error 0.0220867 at the L.Knee marker
Y7 had marker error rms 0.0169768 and max marker error 0.0366843 at the L.Knee marker
Y8 had marker error rms 0.0178472 and max marker error 0.0292494 at the L.ASIS marker
Y9 had marker error rms 0.0164671 and max marker error 0.0326594 at the R.Knee marker
Y10 had marker error rms 0.0172277 and max marker error 0.0365453 at the L.Knee marker
Y11 had marker error rms 0.0159794 and max marker error 0.0285476 at the L.ASIS marker
Y12 had marker error rms 0.0222344 and max marker error 0.0397617 at the R.Heel marker
Y13 had marker error rms 0.0139774 an

In [8]:
ik_df = parse_full_ik_log('/Users/briankeller/Desktop/GRFMuscleModel/Old_Young_Walking_Data/Y_ik_log.txt', trial_names)
abs_max_count, mean_max_count, mean_rms_count = 0, 0 ,0
problem_trials = []
for idx, row in ik_df.iterrows():
    trial = row['trial_name']
    mean_rms = row['mean_rms']
    std_rms = row['std_rms']
    max_rms = row['max_rms']
    mean_max = row['mean_max']
    std_max = row['std_max']
    max_error = row['max_marker_error']
    max_error_marker = row['max_marker_name']
    #print(f'{trial} had mean rms {mean_rms}, std rms {std_rms}, and max rms{max_rms}. The average max error was{mean_max}, with a standard deviation {std_max} and overall max {max_error} at the marker {max_error_marker}')
    if mean_rms > 0.03:
        #print(f'{trial} had mean RMS greater than 3 cm, it was {mean_rms}')
        mean_rms_count += 1
        problem_trials.append(trial)
    elif mean_max > 0.04:
        #print(f'{trial} had mean max error greater than 4 cm, it was {mean_max}')
        mean_max_count += 1
        problem_trials.append(trial)
    # elif max_error > 0.05:
    #     abs_max_count += 1
    #     print(f'{trial} had max_error greater than 6 cm, it was {max_error} at the {max_error_marker} marker')
    #     problem_trials.append(trial)
print(f'{mean_rms_count} trials with Mean RMS > 3 cm')
print(f'{mean_max_count} trials with Mean max error > 4 cm')
print(f'{abs_max_count} trials with overall max error > 6 cm')




Read inverse kinematics log
14 trials with Mean RMS > 3 cm
3 trials with Mean max error > 4 cm
0 trials with overall max error > 6 cm


In [9]:
root_dir = '/Users/briankeller/Desktop/GRFMuscleModel/Old_Young_Walking_Data/'
os.chdir(root_dir)
for subj, data in subject_trials.items():
    model = osim.Model(f'Results/Scaling/{subj}_scaled.osim')
    for trc, forces in zip(data['tracking'], data['forces']):
        inverse_dynamics(root_dir=root_dir, force_data_filepath=forces['output'], tracking_data_filepath=trc['output'], model=model)

[info] Loaded model Y1_scaled from file Results/Scaling/Y1_scaled.osim
[warning] Couldn't find file 'r_pelvis.vtp'.
[warning] Couldn't find file 'l_pelvis.vtp'.
[warning] Couldn't find file 'sacrum.vtp'.
[warning] Couldn't find file 'r_femur.vtp'.
[warning] Couldn't find file 'r_tibia.vtp'.
[warning] Couldn't find file 'r_fibula.vtp'.
[warning] Couldn't find file 'r_patella.vtp'.
[warning] Couldn't find file 'r_talus.vtp'.
[warning] Couldn't find file 'r_foot.vtp'.
[warning] Couldn't find file 'r_bofoot.vtp'.
[warning] Couldn't find file 'l_femur.vtp'.
[warning] Couldn't find file 'l_tibia.vtp'.
[warning] Couldn't find file 'l_fibula.vtp'.
[warning] Couldn't find file 'l_patella.vtp'.
[warning] Couldn't find file 'l_talus.vtp'.
[warning] Couldn't find file 'l_foot.vtp'.
[warning] Couldn't find file 'l_bofoot.vtp'.
[warning] Couldn't find file 'hat_spine.vtp'.
[warning] Couldn't find file 'hat_jaw.vtp'.
[warning] Couldn't find file 'hat_skull.vtp'.
[warning] Couldn't find file 'hat_ribs